In [100]:
import SAP_GUI
import win32com.client
import pandas as pd
import subprocess
from time import sleep
from datetime import datetime
from datetime import timedelta

"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
user="jespinosap"
password="Digital2023*"
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

"""----------------- Search ZD110 -------------------------------------------------------"""
Transsaccion_ZSD110="zsd110"
Varian_FIRME="PEN-FIRME"
Varian_MERCADEO="PEN-MERCADEO"
Varian_DISNAL_TOTAL="PENDIENTES"
provision_ZSD110="/REVISIONPEN"

"""------------------------------------------ START Search ZD110----------------------------------------------------"""
def Search_ZSD110(Transsaccion,variant,provision):
    session.StartTransaction(Transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    Varians_ZSD110=session.findById("wnd[1]/usr/cntlALV_CONTAINER_1/shellcont/shell")
    Rows_Varians_ZSD110=Varians_ZSD110.RowCount
    # List Variants
    List_Varian_ZSD110=[Varians_ZSD110.GetCellValue(i,"VARIANT") for i in range(0,Rows_Varians_ZSD110)] 
    indice=[indice for indice, dato in enumerate(List_Varian_ZSD110) if dato == variant]
    Varians_ZSD110.selectedRows = indice[0]
    session.findById("wnd[1]/tbar[0]/btn[2]").press()
    session.findById("wnd[0]/usr/ctxtPA_LAYOU").text = provision
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    tabla_zsd110=session.findById("wnd[0]/usr/cntlGRID1/shellcont/shell")     #Select table cont
    return(tabla_zsd110)


"""----------------------------------------------CLEAN COLUMN AND DATA-----------------------------------------------------------"""
def Clean_Columns(List):
    Set_Columns=[i.strip() for i in List.columns]   #Alert in list
    clean_column=[i if "Unnamed" in i else None for i in List.columns]
    clean_column = list(filter(lambda x: x is not None, clean_column)) 
    List=List.set_axis(Set_Columns, axis=1).drop(clean_column,axis=1)  #Drop Unammed: 0
    return(List)

"""------------------------------------------FUNCION PARA CAMBIAR A NUMEROS-------------------------------------------------------"""

def Clean_num(x):
    x = float(str(x).strip().replace(',',''))
    return(x)

"""------------------------------------------FUNCION PARA COMPLETAR los 10 con 00 al inicio---------------------------------------"""
def Complete_00(lista):
    Nueva_Lista_Ordenes=[]
    for i in lista:
        while len(str(i))<10:
            i="0"+i
            # print(len(str(i)),i)
        Nueva_Lista_Ordenes.append(str(i))
    return(Nueva_Lista_Ordenes)

def default_column(default_columns,dataframe):     #Parametros (default_columns: Columnas predeterminadas,dataframe:)
    diccionay_default_column={}
    if len(default_columns)==len(dataframe.columns):
        for i in range(0,len(default_columns)):
            diccionay_default_column[dataframe.columns[i]]=default_columns[i]   
        dataframe=dataframe.rename(columns=diccionay_default_column)   #Remplazo las columnas con las de default
    else:
        None   #asumo columnas originales como estandar
    return(dataframe)




In [9]:
"""-----------------------------------BUSCAR y DESCARGAR TRANSSACCION ZD110---------------------------------------------------------------- """
# Pendiente Firme
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar
Search_ZSD110(Transsaccion_ZSD110,Varian_FIRME,provision_ZSD110)
SAP_GUI.Export_TXT2(Name_ZSD110_FIRME,session)

# Pendiente Mercadeo
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar
Search_ZSD110(Transsaccion_ZSD110,Varian_MERCADEO,provision_ZSD110)
SAP_GUI.Export_TXT2(Name_ZSD110_MERCADEO,session)


# Pendiente Total 
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar
tabla_Total=Search_ZSD110(Transsaccion_ZSD110,Varian_DISNAL_TOTAL,provision_ZSD110)
SAP_GUI.Export_TXT2(Name_ZSD110_TOTAL,session)


def Download_ZSD110_Channels(tabla_zsd110,session):
    row_table=tabla_zsd110.RowCount
    List_Channels=[]
    for i in range(0,row_table-1):  #Se resta uno por los subtotales
        channelID=tabla_zsd110.GetCellValue(i,"VTWEG")
        tabla_zsd110.selectedRows = str(i)
        try:
            if channelID=="01":
                session.findById("wnd[0]/tbar[1]/btn[8]").press()
                session.findById("wnd[1]/tbar[0]/btn[45]").press()
                session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[1,0]").select()
                session.findById("wnd[1]/tbar[0]/btn[0]").press()
                session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"   #CAMBIAR RUTA
                session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "Canal"+str(channelID)+"_ENT.txt"
                session.findById("wnd[1]/usr/ctxtDY_FILE_ENCODING").text = "4310"
                session.findById("wnd[1]/tbar[0]/btn[11]").press()
                session.findById("wnd[1]").close()
            session.findById("wnd[0]/tbar[1]/btn[7]").press()   
            session.findById("wnd[1]/tbar[0]/btn[45]").press()
            session.findById("wnd[1]/usr/subSUBSCREEN_STEPLOOP:SAPLSPO5:0150/sub:SAPLSPO5:0150/radSPOPLI-SELFLAG[1,0]").select()
            session.findById("wnd[1]/tbar[0]/btn[0]").press()
            session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"   #CAMBIAR RUTA
            session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "Canal"+str(channelID)+"_PEN.txt"
            session.findById("wnd[1]/usr/ctxtDY_FILE_ENCODING").text = "4310"
            session.findById("wnd[1]/tbar[0]/btn[11]").press()
            session.findById("wnd[1]").close()
            List_Channels.append(channelID)
            print("Se encontro detalle Channel ",channelID)
        except:
            print("No se encontro ",channelID)
    return(List_Channels)

List_Channels=Download_ZSD110_Channels(tabla_Total,session)

In [101]:
import pandas as pd
"""-----------------------------------BUSCAR y DESCARGAR TRANSSACCION ZD110---------------------------------------------------------------- """
# Pendiente Firme
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar

# Pendiente Mercadeo
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar



# Pendiente Total 
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar


In [102]:
# ZSD110 FIRME
url_ZSD110_FIRME="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_FIRME)+".txt"
Data_ZSD110_FIRME=pd.read_csv(url_ZSD110_FIRME,skiprows=1,delimiter="\t")
Data_ZSD110_FIRME=Clean_Columns(Data_ZSD110_FIRME)
Data_ZSD110_FIRME["Lineas"]=Data_ZSD110_FIRME["Lineas"].apply(lambda x:Clean_num(x))


# ZSD110 MERCADE0
url_ZSD110_MERCADEO="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_MERCADEO)+".txt"
Data_ZSD110_MERCADEO=pd.read_csv(url_ZSD110_MERCADEO,skiprows=1,delimiter="\t")
Data_ZSD110_MERCADEO=Clean_Columns(Data_ZSD110_MERCADEO)
Data_ZSD110_MERCADEO["Lineas"]=Data_ZSD110_MERCADEO["Lineas"].apply(lambda x:Clean_num(x))


# ZSD110 Pendiente Total
url_ZSD110_PENDIENTES="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\"+str(Name_ZSD110_TOTAL)+".txt"
Data_ZSD110_PENDIENTES=pd.read_csv(url_ZSD110_PENDIENTES,skiprows=1,delimiter="\t")
Data_ZSD110_PENDIENTES=Clean_Columns(Data_ZSD110_PENDIENTES)
Data_ZSD110_PENDIENTES["Lineas"]=Data_ZSD110_PENDIENTES["Lineas"].apply(lambda x:Clean_num(x))

"""---------------------CAMBIAR INDICES------------------------------------------"""
Channels_Name={1:"MODERNO",2:"TRADICIONAL",7:"TIENDAS DE BELLEZA",8:"PROFESIONAL",12:"INSTITUCIONAL",14:"HARD DISCOUNTERS",15:"ECOMMERCE",19:"ALTERNATIVO",50:"ALMACENES INTERNOS"}


Data_ZSD110_MERCADEO["Can.distr."]=Data_ZSD110_MERCADEO["Can.distr."].replace(Channels_Name)
Data_ZSD110_FIRME["Can.distr."]=Data_ZSD110_FIRME["Can.distr."].replace(Channels_Name)
Data_ZSD110_PENDIENTES["Can.distr."]=Data_ZSD110_PENDIENTES["Can.distr."].replace(Channels_Name)

In [179]:
Data_ZSD110_FIRME
Data_ZSD110_MERCADEO
Data_ZSD110_PENDIENTES

,Can.distr.,Pedidos,Lineas,Unidades,Valor Neto
0,MODERNO,222,554.0,"4,875","34,982,300"
1,TRADICIONAL,32,475.0,"6,830","62,737,400"
2,TIENDAS DE BELLEZA,114,2397.0,"12,677","105,742,700"
3,PROFESIONAL,83,1054.0,"4,635","95,967,800"
4,INSTITUCIONAL,3,8.0,623,"9,345,100"
5,HARD DISCOUNTERS,2,24.0,"6,422","41,112,800"
6,ECOMMERCE,73,230.0,287,"5,488,300"
7,ALTERNATIVO,0,0.0,0,0
8,ALMACENES INTERNOS,0,0.0,0,0
9,NaN,529,4742.0,"36,349","355,376,400"


In [180]:
Consolidado_Clientes=pd.merge(pd.merge(Data_ZSD110_MERCADEO,Data_ZSD110_FIRME,how="right",on="Can.distr."),Data_ZSD110_PENDIENTES,how="right",on="Can.distr.")
Consolidado_Clientes=Consolidado_Clientes.fillna(0)

In [181]:
Consolidado_Clientes_Stand=['Can.distr.', 'Pedidos', 'Lineas', 'Unidades', 'Valor Neto',
       'Pedidos', 'Lineas', 'Unidades', 'Valor Neto', 'Pedidos',
       'Lineas', 'Unidades', 'Valor Neto']

In [182]:
Consolidado_Clientes=default_column(Consolidado_Clientes_Stand,Consolidado_Clientes)

In [197]:
Datos_lineas_clientes=pd.DataFrame([[1,"ECOMMERCE",0],[2,"Cencosud",0],[3,"Éxito Agenda",0],
              [4,"Canal 1",0],[5,"Canales 2,7,8",0],[6,"BAZAR",0],[7,"Mercadeo",0],
              [8,"Canales 14, 19 Citas",0],[9,"Extra agenda",0]],columns=["Prioridad","Clientes","Cantidad"])


In [198]:
valor_ecommerce =Consolidado_Clientes.loc[6][6]   #6 Ecommers  y 6 Lineas
valor_canal_2_7_8=Consolidado_Clientes.loc[1][6]+Consolidado_Clientes.loc[2][6]+Consolidado_Clientes.loc[3][6]   #Tradiconal suma(1,2,3)
valor_Bazar=Consolidado_Clientes.loc[4][6]   
valor_Mercadeo=sum([Consolidado_Clientes.loc[i][2] for i in range(0,len(Consolidado_Clientes)-1)])
valor_canal_14_19=Consolidado_Clientes.loc[5][6]+Consolidado_Clientes.loc[7][6]

In [199]:
Consolidado_Clientes
Datos_lineas_clientes["Cantidad"][0] = valor_ecommerce
Datos_lineas_clientes["Cantidad"][4] = valor_canal_2_7_8
Datos_lineas_clientes["Cantidad"][5] = valor_Bazar
Datos_lineas_clientes["Cantidad"][6] = valor_Mercadeo
Datos_lineas_clientes["Cantidad"][7] = valor_canal_14_19

C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_2764\219365640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Datos_lineas_clientes["Cantidad"][0] = valor_ecommerce
C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_2764\219365640.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Datos_lineas_clientes["Cantidad"][4] = valor_canal_2_7_8
C:\Users\prac.ingindustrial2\AppData\Local\Temp\ipykernel_2764\219365640.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [201]:
Datos_lineas_clientes

,Prioridad,Clientes,Cantidad
0,1,ECOMMERCE,244
1,2,Cencosud,0
2,3,Éxito Agenda,0
3,4,Canal 1,0
4,5,"Canales 2,7,8",3510
5,6,BAZAR,8
6,7,Mercadeo,833
7,8,"Canales 14, 19 Citas",24
8,9,Extra agenda,0
